In [3]:
import json
import pandas as pd

from bs4 import BeautifulSoup

In [6]:
def get_posts(iter_=0):
    
    with open('./KBoard-Backup-20180701.xml', encoding='utf-8') as f:
        content = f.read()
    bs = BeautifulSoup(content, 'xml')
    datas = bs.select('data')
    
    post_datas = [
        data for data in datas
        if data.board_id
        and (data.board_id.text == '7' or data.board_id.text == '4')
        and data.uid
    ]
    
    idx2category = {
        '4': 'photo',
        '7': 'news'
    }
    
    def get_post(data):
        images = [
            data_.file_path.text for data_ in datas
            if data_.content_uid
            and data_.content_uid.text == data.uid.text
        ]
        nonlocal iter_
        post = {
            'id': iter_,
            'category': idx2category[data.board_id.text],
            'date': data.date.text,
            'view_count': data.view.text,
            'thumbnail': data.thumbnail_file.text,
            'writer': data.member_display.text,
            'title': data.title.text,
            'content': data.content.text.replace('\n', '<br>'),
            'images': images
        }
        iter_ += 1
        return post
    
    posts = [get_post(data) for data in post_datas]
    
    return posts

In [7]:
posts = get_posts()

In [8]:
with open('./post.json', 'w') as f:
    json.dump(posts, f)